In [15]:

import numpy as np
import struct  
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn
from sklearn.manifold import TSNE
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
from time import time

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [16]:
#define required function 

#create and store weights
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

#create and store bias
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

#build convolutional layer
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

#max pool
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 3, 3, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

#randomly choose n samples in the dataset to train
def random_batch(n, imgs, labels):
    index = np.arange(0 , len(imgs))
    np.random.shuffle(index)
    idx = index[0:n]
    imgs_random = [imgs[ i] for i in idx]
    labels_random = [labels[ i] for i in idx]

    return np.asarray(imgs_random), np.asarray(labels_random)



In [18]:
#building input holder
x  = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])

x_image = tf.reshape(x, [-1,28,28,1])

#building two convolutional layers with rectified linear activations and max pool function.
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 128])
b_conv2 = bias_variable([128])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

#building two fully connected layers
W_fc1 = weight_variable([7 * 7 * 128, 512])
b_fc1 = bias_variable([512])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 128])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([512, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

#building traning step to reduce cross entropy.
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

#test the accuracy
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

sess = tf.Session()
sess.run(tf.global_variables_initializer())
    

start = time()

#train the model 1000 times, each time ramdonly pick 50 samples as the batch.
for i in range(1000):
    batch = random_batch(50,mnist.train.images,mnist.train.labels)
    #show the traning accuracy every 10 steps.
    if i%10 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    sess.run(train_step,feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
print ("test accuracy %g"% sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 0.5}) )
sess.close()

end = time()
#print the time cost
cost = str((end-start)/60.)+' minutes'
print('Total time cost: '+cost)

step 0, training accuracy 0.24
step 10, training accuracy 0.34
step 20, training accuracy 0.52
step 30, training accuracy 0.6
step 40, training accuracy 0.54
step 50, training accuracy 0.76
step 60, training accuracy 0.66
step 70, training accuracy 0.84
step 80, training accuracy 0.86
step 90, training accuracy 0.76
step 100, training accuracy 0.92
step 110, training accuracy 0.9
step 120, training accuracy 0.86
step 130, training accuracy 0.9
step 140, training accuracy 0.8
step 150, training accuracy 0.82
step 160, training accuracy 0.98
step 170, training accuracy 0.9
step 180, training accuracy 0.92
step 190, training accuracy 0.92
step 200, training accuracy 0.88
step 210, training accuracy 0.96
step 220, training accuracy 0.9
step 230, training accuracy 0.88
step 240, training accuracy 0.94
step 250, training accuracy 0.94
step 260, training accuracy 0.82
step 270, training accuracy 0.9
step 280, training accuracy 0.96
step 290, training accuracy 0.98
step 300, training accuracy 